<a href="https://colab.research.google.com/github/graccelinn/optimization_project1/blob/main/Optimization_Q4_FS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTEAD OF MINIMIZING B-CBAR I WANT TO MINIMIZE THE MAXIMUM MONTHLY B-CVAR ACROSS THE YEAR.

Each day is considered an independent scenario. So longer months naturally contribute more to the estimate because they provide more information.

In [ ]:
# --- prepare assets and arrays (same as before) ---
assets = [c for c in r19.columns if c != INDEX_NAME]
Y = r19[assets].to_numpy()   # shape (q_total, n)
q_total, n = Y.shape
mu = r19[assets].mean().to_numpy()

# --- map each row to a month period (periods preserve order) ---
dates = pd.to_datetime(r19.index)
month_labels = dates.to_period('M')                # e.g. Period('2019-01', 'M')
months = month_labels.unique().tolist()            # list of months in order

# mapping month -> row indices (0-based into Y)
month_to_rows = {m: np.where(month_labels == m)[0].tolist() for m in months}
q_m = {m: len(month_to_rows[m]) for m in months}

# --- Build Gurobi model ---
model = gp.Model("min_max_monthly_cvar")
model.Params.OutputFlag = 0

# decision vars
x = model.addVars(n, lb=0.0, name="x")                    # asset weights (n vars)
alpha = {m: model.addVar(lb=-GRB.INFINITY, name=f"alpha_{str(m)}") for m in months}
u = model.addVars(q_total, lb=0.0, name="u")              # one slack per day
T = model.addVar(lb=-GRB.INFINITY, name="T")             # maximum monthly CVaR to minimize

# budget and mean floor
model.addConstr(gp.quicksum(x[j] for j in range(n)) == 1.0, name="budget")
model.addConstr(gp.quicksum(mu[j] * x[j] for j in range(n)) >= R_FLOOR, name="mean_floor")

# tail constraints: for each day k use the alpha of that day's month
for k in range(q_total):
    m_label = month_labels[k]
    # -Y[k,:]@x - alpha_m - u_k <= 0  <=> u_k >= L_k(x) - alpha_m
    model.addConstr(
        -gp.quicksum(Y[k, j] * x[j] for j in range(n)) - alpha[m_label] - u[k] <= 0.0,
        name=f"tail_day_{k}"
    )

# monthly CVaR definition and link to T
for m_label in months:
    rows = month_to_rows[m_label]
    qm = q_m[m_label]
    if qm <= 0:
        raise RuntimeError(f"Month {m_label} has no observations.")
    cvar_coef = 1.0 / ((1.0 - BETA) * qm)
    # alpha_m + cvar_coef * sum_{k in rows} u_k <= T
    model.addConstr(
        alpha[m_label] + cvar_coef * gp.quicksum(u[k] for k in rows) <= T,
        name=f"cvar_le_T_{str(m_label)}"
    )

# objective: minimize T (the maximum monthly CVaR)
model.setObjective(T, GRB.MINIMIZE)

# solve
model.optimize()
if model.Status != GRB.OPTIMAL:
    raise RuntimeError(f"Gurobi did not find an optimal solution (status {model.Status}).")

# extract weights
w = pd.Series([x[j].X for j in range(n)], index=assets, name="weight")

# --- helper to compute VaR/CVaR from a returns DataFrame grouped by months ---
def var_cvar_array_for_months(weights: pd.Series, returns_df: pd.DataFrame, beta=BETA):
    # returns_df: rows are days with a DatetimeIndex
    dates = pd.to_datetime(returns_df.index)
    month_labels = dates.to_period('M')
    months = month_labels.unique()
    results = {}
    for m in months:
        rows_mask = (month_labels == m)
        Rm = returns_df.loc[rows_mask, weights.index]
        if len(Rm) == 0:
            results[m] = (np.nan, np.nan)
            continue
        pr = Rm.to_numpy() @ weights.to_numpy()   # daily portfolio returns for that month
        L = -pr
        try:
            v = np.quantile(L, beta, method="linear")
        except TypeError:
            v = np.quantile(L, beta, interpolation="linear")
        cvar = float(L[L >= v - 1e-12].mean())
        results[m] = (float(v), cvar)
    return results

# compute in-sample monthly CVaRs (2019)
in_sample_monthly = var_cvar_array_for_months(w, r19[assets])

# report
print("\n=== Minimize max-monthly CVaR result ===")
print("Optimized top weights:\n", w.sort_values(ascending=False).head(10))
print(f"Objective T (minimized max monthly CVaR, in-sample) = {model.ObjVal:.6f}")

# show monthly CVaRs (in-sample) and maximum
in_cvars = pd.Series({m: v[1] for m, v in in_sample_monthly.items()})
out_cvars = pd.Series({m: v[1] for m, v in out_sample_monthly.items()})
print("\nIn-sample (2019) monthly CVaR (as decimals):")
print(in_cvars)
print(f"Max in-sample monthly CVaR = {in_cvars.max():.6f}")